## Dutta and Datta 2016

In [229]:
import pandas
import numpy
from statistics import mean
from scipy import stats

### Function

In [1]:
def dd_wilcoxon(clusters,x,groups):
  data = pandas.DataFrame({'clusters':clusters,'x':x,'groups':groups})
  n = data.shape[0]
  m = len(set(clusters))
  print(m)
  dw = data.loc[data.groups==1].reset_index(drop=True)
  nv = dw.groupby('clusters')['groups'].transform('sum').tolist()
  dw = dw[['clusters','x']].values.T.tolist()
  dw[0] = list(map(int,dw[0]))
  M1 = []
  for cluster,value in zip(dw[0],dw[1]):
    ds1 = data.loc[data.groups==1,['clusters','x']].values.T.tolist()
    vs1 = [1 if z<=value else 0 for z in ds1[1]]
    vs1 = list(zip(ds1[0],vs1))
    sl1 = [mean([v[1] for v in vs1 if v[0] is t]) for t in set([t[0] for t in vs1])]
    ds0 = data.loc[data.groups==0,['clusters','x']].values.T.tolist()
    vs0 = [1 if z<=value else 0 for z in ds0[1]]
    vs0 = list(zip(ds0[0],vs0))
    sl0 = [mean([v[1] for v in vs0 if v[0] is t]) for t in set([t[0] for t in vs0])]
    mm = [(u+v)/2 for u,v in zip(sl1,sl0)]
    mm[cluster] = 0
    M1.append(mm)
  M1 = numpy.array(M1).T / nv
  sFs1 = M1.sum()
  vf1 = []
  for cluster in list(set(clusters)):
    idx = [index for index in range(len(dw[0])) if dw[0][index] == cluster] 
    vf = numpy.delete(M1,idx,axis=1)
    vf = numpy.delete(vf,cluster,axis=0)
    vf = vf.sum()
    vf1.append(vf)
  M2 = []
  for cluster,value in zip(dw[0],dw[1]):
    ds1 = data.loc[data.groups==1,['clusters','x']].values.T.tolist()
    vs1 = [1 if z<value else 0 for z in ds1[1]]
    vs1 = list(zip(ds1[0],vs1))
    sl1 = [mean([v[1] for v in vs1 if v[0] is t]) for t in set([t[0] for t in vs1])]
    ds0 = data.loc[data.groups==0,['clusters','x']].values.T.tolist()
    vs0 = [1 if z<value else 0 for z in ds0[1]]
    vs0 = list(zip(ds0[0],vs0))
    sl0 = [mean([v[1] for v in vs0 if v[0] is t]) for t in set([t[0] for t in vs0])]
    mm = [(u+v)/2 for u,v in zip(sl1,sl0)]
    mm[cluster] = 0
    M2.append(mm)
  M2 = numpy.array(M2).T / nv
  sFs2 = M2.sum()
  vf2 = []
  for cluster in list(set(clusters)):
    idx = [index for index in range(len(dw[0])) if dw[0][index] == cluster] 
    vf = numpy.delete(M2,idx,axis=1)
    vf = numpy.delete(vf,cluster,axis=0)
    vf = vf.sum()
    vf2.append(vf)

  v1 = ((sFs1+sFs2)/4) + (m/2)
  vd = ((numpy.array(vf1)+numpy.array(vf2))/4) + (m-1)/2

  T = v1             
  ET = 0.25*m*(m+1)
  test = v1 - vd
  vtest = numpy.var(test,ddof=1)
  vhat = vtest*numpy.power(m,2)/(m-1)
  Z = (T-ET)/numpy.sqrt(vhat)
  pvalue = stats.norm.sf(abs(Z))
  return(Z,pvalue)
